In [91]:
import pandas as pd
import numpy as np
from collections import Counter
import operator
import time
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

In [92]:
train = pd.read_csv('./../src/data/train.txt', sep='|')
labels = train.CLASE
test = pd.read_csv('./../src/data/test.txt', sep='|')

train = train.drop(['ID'], axis=1)

In [93]:
train['CADASTRALQUALITYID'] = train['CADASTRALQUALITYID'].astype('category')

In [94]:
df = pd.DataFrame({"nº NANs": train.isna().sum(), "% NANs": round((train.isna().sum()/train.shape[0])*100,1)})
df

,nº NANs,% NANs
X,0,0.0
Y,0,0.0
Q_R_4_0_0,0,0.0
Q_R_4_0_1,0,0.0
Q_R_4_0_2,0,0.0
Q_R_4_0_3,0,0.0
Q_R_4_0_4,0,0.0
Q_R_4_0_5,0,0.0
Q_R_4_0_6,0,0.0
Q_R_4_0_7,0,0.0


Imputación provisional de NANs:
- Categóricas: Nueva categoría 'Unknown'
- Numéricas: Mediana

In [95]:
train = train.drop(['CLASE'], axis=1)
for column in train:
    if train[column].dtype.name == 'category':
        train[column].cat.add_categories('Unknown', inplace=True)
        train[column].fillna('Unknown', inplace=True)
    elif train[column].dtype.name != 'category':
        train[column].fillna(train[column].median(), inplace=True)


In [108]:
train_y = pd.concat([train,labels], axis=1)

In [109]:
train_y.shape

(13057, 55)

In [110]:
# Cálculo de porcentaje de correspondencia entre cada valor único de una variable determinada y la clase:
def correspondencia_valor_clase(train_y,var):
    
    # Porcentajes:
    
    clases = train_y.CLASE.unique()
    values = train_y[var].unique()
    
    porcentajes = {}
    for v in values:
        total = train_y.loc[(train_y.loc[:,var] == v)].shape[0]

        aux = []
        for c in clases:
            #print(col_name,' , ',v,' , ',c,' , ',round(((train_y.loc[(train_y.loc[:,col_name] == v) & (train_y.loc[:,'status_group'] == c)].shape[0])/total)*100,2), '%')
            aux.append(round(((train_y.loc[(train_y.loc[:,var] == v) & (train_y.loc[:,'CLASE'] == c)].shape[0])/total)*100,2)) 
        porcentajes[v] = aux
    
    # Diferencias:
    
    diferencias = {}
    for i in range(0,values.shape[0]):
        if i < (values.shape[0]-1):  
            for j in range(i+1,values.shape[0]):
                name = values[i]+' - '+values[j]
                diferencias[name] = round(float(sum(abs(pd.DataFrame(porcentajes[values[i]]).values-pd.DataFrame(porcentajes[values[j]]).values))),2)
    
    return porcentajes,diferencias

In [111]:
p,d = correspondencia_valor_clase(train_y,'CADASTRALQUALITYID')

UFuncTypeError: ufunc 'add' did not contain a loop with signature matching types (dtype('<U32'), dtype('<U32')) -> dtype('<U32')

In [67]:
pd.DataFrame(sorted(d.items(), key=operator.itemgetter(1), reverse=True)).tail(60)

,0,1
18,2 - A,96.47
19,4 - A,95.07
20,7 - A,93.88
21,C - 1,93.07
22,6 - A,92.56
23,5 - C,90.49
24,8 - C,87.80
25,1 - A,87.13
26,5 - A,85.95
27,8 - A,84.24


In [68]:
# waterpoint_type_group

train['CADASTRALQUALITYID'] = train['CADASTRALQUALITYID'].astype('object')

ids =np.where(train['CADASTRALQUALITYID']=='2')[0]
train.loc[ids,'CADASTRALQUALITYID'] = 'a'
ids =np.where(train['CADASTRALQUALITYID']=='3')[0]
train.loc[ids,'CADASTRALQUALITYID'] = 'b'
ids =np.where(train['CADASTRALQUALITYID']=='4')[0]
train.loc[ids,'CADASTRALQUALITYID'] = 'c'
ids =np.where(train['CADASTRALQUALITYID']=='1')[0]
train.loc[ids,'CADASTRALQUALITYID'] = 'd'
ids =np.where(train['CADASTRALQUALITYID']=='6')[0]
train.loc[ids,'CADASTRALQUALITYID'] = 'e'
ids =np.where(train['CADASTRALQUALITYID']=='7')[0]
train.loc[ids,'CADASTRALQUALITYID'] = 'f'
ids =np.where(train['CADASTRALQUALITYID']=='5')[0]
train.loc[ids,'CADASTRALQUALITYID'] = 'g'
ids =np.where(train['CADASTRALQUALITYID']=='8')[0]
train.loc[ids,'CADASTRALQUALITYID'] = 'h'
ids =np.where(train['CADASTRALQUALITYID']=='9')[0]
train.loc[ids,'CADASTRALQUALITYID'] = 'i'
ids =np.where(train['CADASTRALQUALITYID']=='B')[0]
train.loc[ids,'CADASTRALQUALITYID'] = 'j'
ids =np.where(train['CADASTRALQUALITYID']=='A')[0]
train.loc[ids,'CADASTRALQUALITYID'] = 'k'
ids =np.where(train['CADASTRALQUALITYID']=='C')[0]
train.loc[ids,'CADASTRALQUALITYID'] = 'l'
ids =np.where(train['CADASTRALQUALITYID']=='Unknown')[0]
train.loc[ids,'CADASTRALQUALITYID'] = 'm'

train['CADASTRALQUALITYID'] = train['CADASTRALQUALITYID'].astype('category')

In [96]:
for column in train:
    if train[column].dtype.name == 'category':
        train[column] = LabelEncoder().fit_transform(train[column])

Normalizar:

(x-mean)/std

In [ ]:
for column in train:
    train[column] = (train[column] - train[column].mean()) / train[column].std()

Rango (0,1)

In [97]:
for column in train:
    train[column] = train[column]/max(train[column])

In [72]:
def validacion_cruzada(modelo, X, y, cv):
    y_test_all = []
    suma = 0
    for train, test in cv.split(X, y):
        t = time.time()
        modelo = modelo.fit(X[train],y[train])
        y_pred = modelo.predict(X[test])
        tiempo = time.time() - t
        #print("Score: {:.4f}, tiempo: {:6.2f} segundos".format(accuracy_score(y[test],y_pred) , tiempo))
        y_test_all = np.concatenate([y_test_all,y[test]])
        suma = suma + accuracy_score(y[test],y_pred)
    suma = suma/3
    print("---------------------------- >>> Media:",suma)
    print("")

In [105]:
X = train.values
y = np.ravel(labels.values)

In [106]:
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=123456)
k_values = (21,23,25,27,29,31,33,35,37)
for k in k_values:
    knn = KNeighborsClassifier(n_jobs=-1,n_neighbors=k,weights='distance',metric='manhattan')
    print("=========> Numero de vecinos: ",k,"manhattan")
    validacion_cruzada(knn,X,y,skf)

=========> Numero de vecinos:  21 manhattan
---------------------------- >>> Media: 0.5566362349270054

=========> Numero de vecinos:  23 manhattan
---------------------------- >>> Media: 0.5574020431311739

=========> Numero de vecinos:  25 manhattan
---------------------------- >>> Media: 0.5567891396737882

=========> Numero de vecinos:  27 manhattan
---------------------------- >>> Media: 0.5558700044312465

=========> Numero de vecinos:  29 manhattan
---------------------------- >>> Media: 0.5547212657317694

=========> Numero de vecinos:  31 manhattan
---------------------------- >>> Media: 0.5534958987040716

=========> Numero de vecinos:  33 manhattan
---------------------------- >>> Media: 0.5528066132552263

=========> Numero de vecinos:  35 manhattan
---------------------------- >>> Media: 0.5514280247620528

=========> Numero de vecinos:  37 manhattan
---------------------------- >>> Media: 0.5504324371461197



In [100]:
train = pd.concat([train, labels], axis=1)


In [101]:
train = train[train.CLASE != 'RESIDENTIAL']

In [102]:
labels = labels[labels != 'RESIDENTIAL']

In [103]:
labels.shape

(13057,)

In [104]:
train = train.drop(['CLASE'], axis=1)